In [0]:
from pyspark.sql.functions import col, current_timestamp

#  Define the paths for volume and autoloader
input_path = "/Volumes/workspace/default/nhs_workforce_raw_uploads"
checkpoint_path = "/Volumes/workspace/default/nhs_workforce_raw_uploads/_checkpoints/bronze_ingestion"
#dbutils.fs.rm(checkpoint_path, True)

#Set up the Auto Loader Stream
bronze_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", checkpoint_path)
    .option("header", "true")
    .load(input_path))



# Renaming to remove spaces and standardize names
df_clean = bronze_stream.withColumn("ingested_at", current_timestamp())\
      .withColumnRenamed("Org Code", "OrgCode") \
      .withColumnRenamed("Org Name", "OrgName") \
      .withColumnRenamed("NHSE region code" , "NHSERegionCode") \
      .withColumnRenamed("NHSE region name", "NHSERegionName") \
      .withColumnRenamed("ICS code", "ICSCod") \
      .withColumnRenamed("ICS name", "ICSName") \
      .withColumnRenamed("Cluster group", "ClusterGroup") \
      .withColumnRenamed("Benchmark group", "BenchmarkGroup") \
      .withColumnRenamed("Staff Group", "StaffGroup") 


#Write the stream to your Bronze Table
(df_clean.writeStream
    .option("checkpointLocation", checkpoint_path)
    .trigger(availableNow=True) # Important for Free Edition: processes files and stops
    .outputMode("append")
    .table("nhs_bronze_workforce"))